In [1]:
# coding: utf-8

# # Investigating QSOs near Cen A (NGC 5128)
# [Simbad Information for NGC 5128](http://simbad.u-strasbg.fr/simbad/sim-basic?Ident=cen+a&submit=SIMBAD+search)


## Very most basic/standard imports:
import matplotlib.pyplot as plt
import numpy as np

##If you read FITS or ascii tables, these are necessary:
from astropy.io import ascii
from astropy.table import QTable

##Automatic unit tracking...maybe?
import astropy.units as u

# AstroPy treatment of coordinates
from astropy.coordinates import SkyCoord

# SIMBAD query functionality
from astroquery.simbad import Simbad

# Set up the resolution
import matplotlib as mpl
mpl.rc("savefig", dpi=300)

In [3]:
# ### Grab Cen A information from SIMBAD
cenA = Simbad.query_object('ngc5128')

# Fill `SkyCoord` variables.
cenAdistance = 3.8 * u.Mpc  # Harris et al. (2010)
cenACoords = SkyCoord(cenA['RA'][0], cenA['DEC'][0], unit=(u.hourangle, u.deg))

########################################
# Gamma/radio emission in JPEG

# Load in PR image!
#imCenA = plt.imread('cenA_lobes.jpg')     #
imCenA = plt.imread('cenA-Fermi-noLabel.jpg')
imCenA = imCenA[:,0:556,:]



In [4]:
# Work out angle for FoV on the sky. The "scale" is based on a measurement with
# a different image, so need to get that scale right (hence all the bits at the
# end).
scale = 300.*u.kpc / (595*786./1007) # kpc/pixel
FoV = np.array(np.shape(imCenA)[0:-1])
# Put things in the right order
FoV = FoV[[1,0]]
FoV_kpc = FoV * scale
angle = np.rad2deg(np.arctan(FoV_kpc / cenAdistance.to('kpc'))  )
area = angle[0]*angle[1]                  # x * y

In [6]:
###############
## Apertures sizes
aperture = [4, 6, 9, 15]

## Add QSOs
dr7qso = QTable.read('data/dr7_qso_galex.fits')
zem = dr7qso['Z'][0]
fuv = dr7qso['FUV_MAG'][0]
nuv = dr7qso['NUV_MAG'][0]

limitingmags = [18.3,19.2,20.1,21.8]
numQSOs = np.zeros_like(limitingmags)
for j in np.arange(np.size(limitingmags)):
    num = ((fuv > 10) & (fuv < limitingmags[j]))
    num = num.sum()
    numQSOs[j] += num

# Make this a relative number:
numQSOs = np.array(numQSOs)/np.max(numQSOs)

# If we want it, the SDSS spectroscopic area
SDSS_spec_area = 9380.

# Rough QSO density for 15-m
qso_density_ref = 5./u.deg**2


In [7]:
for j in np.arange(np.size(limitingmags)):
    ######################################################################
    ## Set up the figure:
    plt.figure(figsize=(2.47,3.36))

    ###############
    ## Image
    ax = plt.subplot(111)

    ax.imshow(imCenA, zorder=0)

    # make these tick labels invisible
    plt.setp(ax.get_xticklabels(), visible=False);
    plt.setp(ax.get_yticklabels(), visible=False);

    # Get rid of tick marks
    ax.tick_params(axis=u'both', width=0)
    ax.minorticks_off()


    num_qsos = qso_density_ref * numQSOs[j] * area

    xqso=np.random.randint(0,FoV[0],size=num_qsos)
    yqso=np.random.randint(0,FoV[1],size=num_qsos)

    QSOcolor = '#FFCC66'
    plt.plot(xqso,yqso,'*',color='white',zorder=1,
                 markersize=4,alpha=1,
                 markeredgecolor=QSOcolor,markeredgewidth=0.25)


    credit_text = 'Image credit: NASA/DOE/Fermi LAT Collaboration, Capella Observatory'
    plt.text(0,FoV[1]+5,credit_text,style='italic',
                 color='k',
                 fontsize=3.5,
                 va='top')

    # Box
    xbox = [450,FoV[0]-1]
    y1 = 125
    y2 = 0
    plt.fill_between(xbox,y1,y2,color='k',zorder=2)

    # Explanatory text
    xpos = np.mean(xbox)
    plt.text(xpos, 10, 'Optical',color='#3E77FF',
                 fontsize=6,
                 ha='center',va='top')
    plt.text(xpos, 55, r'$\gamma$-ray',color='#F900E0',
                 fontsize=6,
                 ha='center',va='top')
    plt.text(xpos, 100, 'Radio',color='#FD8731',
                 fontsize=6,
                 ha='center',va='top')



    cenACoords = [278,400]
    cenAradius = 595*786./1007/2.
    impactCircle = mpl.patches.Circle((cenACoords),cenAradius,
                                          color='w', linestyle=':', fill=False,
                                          zorder=3)
    ax.add_artist(impactCircle)
    # Label the impact parameter:
    xx=278+0.78*cenAradius
    yy=400.+0.78*cenAradius
    ax.text(xx,yy,r'$150$ kpc',
                fontsize=6,fontstyle='italic',
                rotation = 45,
                horizontalalignment='center',verticalalignment='center',
                color='white',zorder=3)

    # Put a title on it:
    plt.title('{0}-m'.format(aperture[j]))

    # Write the figure to PDF
    output_name = 'fig-CenA_LUVOIR_{0}m.pdf'.format(aperture[j])
    plt.savefig(output_name)
    plt.close()


/Users/tumlinson/yt-conda/lib/python2.7/site-packages/ipykernel/__main__.py:23: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/Users/tumlinson/yt-conda/lib/python2.7/site-packages/ipykernel/__main__.py:24: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
